In [1]:
source("base/it-402-dc-common_vars.r")

── Attaching packages ────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ───────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


── Column specification ───────────────────────────────────────────────────────────────────
cols(
  QualificationId = col_character(),
  QualificationTitle = col_character(),
  SCQFLevel = col_character(),
  DataStartYear = col_character(),
  DataEndYear = col_character()
)




In [2]:
#library(tidyverse) - called in common_vars
library(ggtext)

library(DBI)


In [3]:
#sessionInfo() 
#
#if ("package:scales" %in% search())
#    detach("package:scales", unload = TRUE)
#
#sessionInfo()

## Notes


#### Legal (ISO) gender types:

* https://data.gov.uk/education-standards/sites/default/files/CL-Legal-Sex-Type-v2-0.pdf


#### From 2010 and all data stored as % 

* need to relax sum to 100%
* 			
Symbol	Meaning		
    * '-'	Not Applicable		
    * '-'	No Entries (Table 3)		
    * 0%	Less than 0.5%		
    * ***	Fewer Than 5 Entries	
    
<br />


#### *** Data stored in percentages only from 2014 ***
<br />



#### Advanced Higher
* Craft and Design changed to Product Design in 2004
* Accounting and Finance changed to Accounting in 2005

#### Higher
* Accounting and Finance changed to Accounting in 2004		
* Craft and Design changed to Product Design in 2004.		
* Tourism changed to Travel and Tourism in 2005.	

<br />&nbsp;<br />


<br /> &nbsp;<br />
<h3>1986-8</h3>

* Standard Grades only

* Ordinary Grade, Higher &amp; CSYS did not include Computing

<br />

* __NoAward__ calculated as diff b/n all entries and grades awarded. The value represents both those who did not achieve a named grade or did not take the assessment but were not withdrawn

<br />


* From 1999 to ?
    * Awards in Optional Papers, Music Practical Harmony and Music Parts II - VI are not expressed in terms of bands.



<br />&nbsp;<br />

# 2020

* uses original results - because revision in Aug and update Dec do not show include distribution by gender.
* for comparison along grades overall Dec revision will be used
* however, __note__ grade results not comparable to previous years due to COVID assessment. This will be repeated for 2021 results 
<br />

* __Notes in original spreadsheet__ 
    * Grade D was widened in 2020.  As a consequence, Grade D attainment (and No Award results) from 2018/19 and previous years are not directly comparable.
        * In data this is recorded as D*
    * Attainment of any candidate that has not identified as Male or Female will not feature in the published sex tables due to the low numbers involved, in line with SQA standard disclosure control rules.
    

## Population &ndash; Candidate Counts

#### Notes

* from 2014	At Higher levels, all passes === sum of grades A to C; however D appears to be considered a pass in at least the 2020 data

* school rolls - "There are slight differences between published totals and the figures obtained from summing the dataset in this spreadsheet."
    * the calculations here use sums, i.e., disregard the published totals

## Notes &ndash; Teacher Censuses

* In techer census data
Symbol	Meaning		
    * '..'   =    not available
    * '#'    =    not applicable
    * '-'    =    nil or rounds to nil
    * '*'    =    disclosive data

* Prior to 2020, subject not known was grouped under other 
* TopSubjects used for column label where - FTE for the teacher post is used for both main and other subject, therefore if an individual teaches a main and other subject the post FTE figure is used for both


In [4]:
# themes commented out only because not currently needed or swapping between values

dressCodeTheme <- 
        theme_bw() +
        #theme_void() + 
        theme(panel.grid.major.y = element_line(), #panel.grid.minor.y = element_blank(),
                panel.grid.major.x = element_blank(), #panel.grid.minor.x = element_blank(), 
                axis.title = element_text(size = 16), 
                axis.text.x = element_text(size = 12), # angle = 0, vjust = 0.3),
                axis.text.y = element_markdown(size = 12), #element_text(size = 14),
                plot.title = element_text(size = 20),
                legend.title = element_text(size = 14), legend.text = element_text(size = 12), 
                strip.text = element_text(size = 18)
            ) 

In [5]:
# evaluation will fail if focus_subject not set in advance
if (!exists("focus_subject") || is_null(focus_subject) || (str_trim(focus_subject) == "")) 
    focus_subject <- "computing"


formatted_subject_labels_as_expr <- expression(
    formatted_subject_labels <- paste0(
        "<span style = 'color: ",
             if_else(str_detect(Subject, regex(paste0(focus_subject, collapse = "|"), ignore_case = TRUE)), default_highlights["focus"], default_highlights["other"]),
         "; font-weight: ",
             if_else(str_detect(Subject, regex(paste0(focus_subject, collapse = "|"), ignore_case = TRUE)), "bold", "normal"), # has no effect altho' docs say it's recognised :(...
         ";'>",
             if_else(str_detect(Subject, regex(paste0(focus_subject, collapse = "|"), ignore_case = TRUE)), "<b>", ""),

                 Subject,
             if_else(str_detect(Subject, regex(paste0(focus_subject, collapse = "|"), ignore_case = TRUE)), "</b>", ""),
         "</span>")
    )
formatted_subject_labels_as_expr



select_focus_subjects_as_expr <- expression(
    select_focus_subjects <- 
        as.factor(if_else(str_detect(Subject, regex(paste0(focus_subject, collapse = "|"), ignore_case = TRUE)), "focus", "other"))
    )
select_focus_subjects_as_expr


expression(formatted_subject_labels <- paste0("<span style = 'color: ", 
    if_else(str_detect(Subject, regex(paste0(focus_subject, collapse = "|"), 
        ignore_case = TRUE)), default_highlights["focus"], default_highlights["other"]), 
    "; font-weight: ", if_else(str_detect(Subject, regex(paste0(focus_subject, 
        collapse = "|"), ignore_case = TRUE)), "bold", "normal"), 
    ";'>", if_else(str_detect(Subject, regex(paste0(focus_subject, 
        collapse = "|"), ignore_case = TRUE)), "<b>", ""), Subject, 
    if_else(str_detect(Subject, regex(paste0(focus_subject, collapse = "|"), 
        ignore_case = TRUE)), "</b>", ""), "</span>"))

expression(select_focus_subjects <- as.factor(if_else(str_detect(Subject, 
    regex(paste0(focus_subject, collapse = "|"), ignore_case = TRUE)), 
    "focus", "other")))

<br />&nbsp;<br />

# Functions for visual exploration pipeline

<br />

In [3]:
loadAndParseFile <- 
    function(pathTofile, glimpseContent = F, qualificationType = NULL, qualificationYear = NULL) {
        
        print(paste0("Parsing qualifications file '", pathTofile, "'"))
        
        if (is_null(qualificationType) || (str_trim(qualificationType) == "")) {
            
            qualificationType <- basename(pathTofile)

            if (!str_detect(qualificationType, regex("(?>.*_)(?>.*\\.)"))) # order matters
                stop("You must specify qualification type!")
                
            qualificationType <- unlist(str_split(qualificationType, "_"))[2]
            qualificationType <- unlist(str_split(qualificationType, "\\."))[1]
            
            if (tolower(qualificationType) %in% tolower(sqa_qualification_list$QualificationId)) #relax on case only
                message(paste0("Qualification type '", qualificationType, "' derived from file name."))
            else
                stop("You must specify qualification type!")
        } 
                
        latestQualificationYear <- year(Sys.Date())
        if (is_null(qualificationYear)) {
            
            qualificationYear <- suppressWarnings(parse_number(basename(pathTofile)))
            if (is.na(qualificationYear) || 
                !(as.integer(qualificationYear) == qualificationYear) || !between(qualificationYear, base_year, latestQualificationYear))
                stop(paste("You must specify qualification year as 'yyyy', between", base_year, "and", latestQualificationYear))
            else
                message(paste0("Qualification year '", qualificationYear, "' derived from file name."))
            
        } else if (!(as.integer(qualification_year) == qualification_year)|| !between(qualificationYear, base_year, latestQualificationYear))
            stop(paste("You must specify qualification year as 'yyyy', between", base_year, "and", latestQualificationYear))
        
        
        # passed tests - load data and return
        award_data <- read_csv(pathTofile, trim_ws = T) %>%
                            filter(rowSums(is.na(.)) != ncol(.)) %>%
                            distinct(across(everything()), .keep_all = TRUE) %>%
                            suppressMessages
        
        award_data <- award_data %>%
            mutate(year = factor(qualificationYear),
                   qualification = factor(qualificationType)) %>%
            select(qualification, year, everything())
        
        
        invisible(award_data)
    }

In [7]:
wrangleData <-
    function(awardData) {
                                
        if (!exists("redundant_column_flags") || is.null(redundant_column_flags)) 
            redundant_column_flags <- c("-percentage*", "-COMP", "-PassesUngradedCourses")
        
        if((awardData %>%
                select(-matches(redundant_column_flags)) %>% # "-percentage")) %>%
                select(matches(c("male-", "female-", "NA-", "NotKnown-"))) %>%

                summarise(data_as_counts = (ncol(.) == 0))
        ) == TRUE)
            awardData <- preProcessDataAsPercentages(awardData)

        
        awardData <- awardData %>%

            mutate_at(c("Subject"), as.factor) %>%
            mutate_at(c("NumberOfCentres"), as.integer) %>%
            rename_with(~ gsub("NA-", "NotApplicable-", .x, fixed = TRUE)) %>%

            mutate_at(vars(starts_with("male-") | starts_with("female-") | starts_with("NA-") | starts_with("NotKnown-") | starts_with("all-")), as.character) %>%
            mutate_at(vars(starts_with("male-") | starts_with("female-") | starts_with("NA-") | starts_with("NotKnown-") | starts_with("all-")), parse_number) %>%
            mutate_at(vars(starts_with("male-") | starts_with("female-") | starts_with("NA-") | starts_with("NotKnown-") | starts_with("all-")), as.integer) %>%
            suppressWarnings

        
        if ((awardData %>%

                select(matches(c("-NoAward"))) %>%
                summarise(no_award_recorded = (ncol(.) == 0)) 
            ) == TRUE) {

            tmp_df <- awardData %>%
                select(-matches(redundant_column_flags)) 
            tmp_df[is.na(tmp_df)] <- 0

            awardData <- bind_cols(awardData,

                tmp_df %>%
                    mutate("male-NoAward" = (`male-Entries` - reduce(select(., (starts_with("male") & !ends_with("-Entries"))), `+`)), # rowSums(select(., starts_with("male-")))
                           "female-NoAward" = (`female-Entries` - reduce(select(., (starts_with("female") & !ends_with("-Entries"))), `+`)), 
                           "all-NoAward" = (`all-Entries` - reduce(select(., (starts_with("all") & !ends_with("-Entries"))), `+`))) %>%
                    select(c("male-NoAward", "female-NoAward", "all-NoAward"))
                ) %>%

                relocate(`male-NoAward`, .after = `male-Entries`) %>%
                relocate(`female-NoAward`, .after = `female-Entries`) %>%
                relocate(`all-NoAward`, .after = `all-Entries`) 
        }
        

        if ("NA-Entries" %in% names(awardData)) {
            awardData <- awardData %>%
                mutate("NA-NoAward" = (`NA-Entries` - reduce(select(., (starts_with("NA") & !ends_with("-Entries"))), `+`))) %>% 
                relocate(`NA-NoAward`, .after = `NA-Entries`)
        } 
        if ("NotKnown-Entries" %in% names(awardData)) {
            awardData <- awardData %>%
                mutate("NotKnown-NoAward" = (`NotKnown-Entries` - reduce(select(., (starts_with("NotKnown") & !ends_with("-Entries"))), `+`))) %>%
                relocate(`NotKnown-NoAward`, .after = `NotKnown-Entries`)
        }
                
        awardData <- awardData %>%

            pivot_longer(!c(qualification, year, Subject, NumberOfCentres), names_to = "grade", values_to = "NoOfStudents") %>%
            separate("grade", c("gender", "grade"), extra = "merge") %>%
            mutate_at(c("gender", "grade", "year"), as.factor)

        invisible(awardData)
    }

In [8]:
preProcessDataAsPercentages <-
    function(awardData) {
        
        awardData <- awardData %>% 

            mutate_at(vars(starts_with("all-") | contains("-percentage")), as.character) %>%
            mutate_at(vars(starts_with("all-") | contains("-percentage")), parse_number) %>%
            mutate_at(vars(starts_with("all-") | contains("-percentage")), as.integer) %>%
            suppressWarnings
         
        includeOpenGender <- ("NA-percentage" %in% names(awardData))
        includeGenderNotKnown <- ("NotKnown-percentage" %in% names(awardData))

        
        awardData <- awardData %>%

            select(Subject, `all-Entries`, ends_with("-percentage")) %>%
            mutate_at(vars(ends_with("-percentage")), ~round(. /100 * `all-Entries`)) %>%

            select_all(~str_replace(., "-percentage", "-Entries")) %>%

            left_join(awardData %>%
                      select(!ends_with("-percentage"))
                     ) %>%

            mutate_at(vars(starts_with("male-") & contains("-percentage")), ~round(. /100 * `male-Entries`)) %>%
            mutate_at(vars(starts_with("female-") & contains("-percentage")), ~round(. /100 * `female-Entries`)) %>%
            mutate_at(vars(starts_with("all-") & contains("-percentage")), ~round(. /100 * `all-Entries`))
        
            if (includeOpenGender)
                awardData <- awardData %>%
                    mutate_at(vars(starts_with("NA-") & contains("-percentage")), ~round(. /100 * `NA-Entries`))
            if (includeGenderNotKnown)
                awardData <- awardData %>%
                    mutate_at(vars(starts_with("NotKnown-") & contains("-percentage")), ~round(. /100 * `NotKnown-Entries`)) 

        
        invisible(awardData %>%
                    select_all(~str_replace(., "-percentage", "-")) 
                 )
    }

In [1]:
writeToDataStore <-
    function(awardData, dbConnection, dbTable = "sqa_data", overwriteDataStore = FALSE) {
        
        if (is_null(awardData) || (nrow(awardData) == 0))
            stop("You must read in qualification data (in a dataframe) containing at least one row!")
        
        table_exists <- as.logical(dbGetQuery(dbConnection, paste0("SELECT COUNT(*) FROM sqlite_master WHERE name = '", 
                                                                   dbTable, "' and type = 'table'")))
        
        if (!table_exists | overwriteDataStore)
            currentRowCount <- 0
        else
            currentRowCount <- dbGetQuery(dbConnection, 
                                          paste("SELECT COUNT(*) FROM", dbTable, "LIMIT 6")) %>%
                                    as.integer
        
        
        if (!table_exists) 
            dbWriteTable(dbConnection, dbTable, awardData)
        else 
            dbWriteTable(dbConnection, dbTable, awardData, append = !overwriteDataStore, overwrite = overwriteDataStore)
        
        # todo - dump duplicates - using dbWriteTable means no primary key set
        
        
        invisible(as.integer(dbGetQuery(dbConnection, paste("SELECT COUNT(*) FROM", dbTable))) - currentRowCount)
    }


In [2]:
removeDuplicatesFromDataStore <- 
    function(dbConnection = NULL, dbTable = NULL) {
        
        if (is_null(dbConnection) | is_null(dbTable))
            stop("You must specify database connection and table to deduplicate!")

        
        if (!as.logical(dbGetQuery(dbConnection, 
                                   paste0("SELECT COUNT(*) FROM sqlite_master WHERE name = '", dbTable, "' and type = 'table'"))))
            stop("No changes made; source table does not exist!")
        
        
        row_count <- as.integer(dbGetQuery(dbConnection, paste("SELECT COUNT(*) FROM", dbTable)))
        if (row_count == 0) {
            
            message(paste0("Table'", dbTable, "' empty; no changes made."))
            invisible(row_count) # nothing to do - allows simple conversion to FALSE
        }
        
        
        tmp_table <- paste0(dbTable, "_tmp")
        if (as.integer(dbGetQuery(dbConnection, 
                                  paste0("SELECT COUNT(*) FROM sqlite_master WHERE name = '", tmp_table, "' and type = 'table'"))) > 0) 
            dbRemoveTable(dbConnection, tmp_table)

        dbExecute(dbConnection, paste("CREATE TABLE ", tmp_table, " AS SELECT * FROM", dbTable, "WHERE 0"))
        dbExecute(dbConnection, paste("INSERT INTO", tmp_table,
                                      "SELECT DISTINCT * FROM", dbTable)
                 )
        
        row_count <- row_count - as.integer(dbGetQuery(dbConnection, paste("SELECT COUNT(*) FROM", tmp_table)))
        if (row_count > 0) {
            dbRemoveTable(dbConnection, dbTable)
            dbExecute(dbConnection, paste("ALTER TABLE", tmp_table, "RENAME TO", dbTable))
        } else
            dbRemoveTable(dbConnection, tmp_table)


        invisible(row_count) # no. of rows deleted
    }


In [11]:
runPipeline <-
    function(dataFile, dbConnection = NULL, overwriteDataStore = FALSE) {
        
        #print(dataFile)
        
        award_data <- loadAndParseFile(dataFile)
        #print(head(award_data))      

        award_data <- wrangleData(award_data) 
        #print(dim(award_data))
        #print(head(award_data))

        if (!is_null(dbConnection))
            writeToDataStore(award_data, dbConnection, overwriteDataStore = overwriteDataStore)
    }


In [12]:
filterInput <-
    function(pathTofile, startYear = NULL, endYear = NULL, qualificationTypes = NULL) {
                
        latestQualificationYear <- year(Sys.Date())
        yearErrorMessage <- paste("Start and end years if set must be between 1986 and", latestQualificationYear,
                                  "(inclusive), and specified as 'yyyy'")
       
        if (is_null(startYear)) 
            startYear <- 1986

        else {             
            startYear <- suppressWarnings(parse_integer(as.character(startYear)))
            if (is.na(startYear) | (startYear > latestQualificationYear))
                stop(yearErrorMessage)            
        } 
        
        if (is_null(endYear)) 
            endYear <- latestQualificationYear

        else {             
            endYear <- suppressWarnings(parse_integer(as.character(endYear)))
            if (is.na(endYear) | !between(endYear, 1986, latestQualificationYear))
                stop(yearErrorMessage)            
        } 


        
        qualificationType <- basename(pathTofile)
        if (is_null(qualificationTypes))
            qualificationTypes <- sqa_qualification_list$QualificationId
        else {
            qualificationTypes <- as.vector(qualificationTypes) # allow single value as input
        
            if (!sum(tolower(qualificationTypes) %in% tolower(sqa_qualification_list$QualificationId)) == length(qualificationTypes)) {
                warning(paste("Non-valid (SQA) qualification types excluded:",
                             qualificationTypes[!tolower(qualificationTypes) %in% tolower(sqa_qualification_list$QualificationId)]))

                qualificationTypes <- qualificationTypes[tolower(qualificationTypes) %in% tolower(sqa_qualification_list$QualificationId)]
            }
        }


        if (!str_ends(qualificationType, ".csv") & !str_detect(qualificationType, regex("(?>.*_)(?>.*\\.)"))) # order matters
            stop("File name must be in format 'yyyy_QualificationType.csv' (e.g., '1999_AdvancedHigher.csv') to apply auto-filter!")
        
        qualificationType <- unlist(str_split(qualificationType, "_"))[2]
        qualificationType <- unlist(str_split(qualificationType, "\\."))[1]

        if (!(tolower(qualificationType) %in% tolower(qualificationTypes)))
            return(FALSE)
                                    
        qualificationYear <- suppressWarnings(parse_number(basename(pathTofile)))
        if (is.na(qualificationYear) | !between(qualificationYear, startYear, endYear))        
            return(FALSE)

        
        # if get this far
        return(TRUE)
    }


In [13]:
# preferable to open connection in each file reusing code?

#dbConn <- dbConnect(RSQLite::SQLite(), paste0(path_to_file_store, "/sqa-data-db.sqlite"))
#dbConn


In [14]:
# sample - edit and run as needed

## test run
#dbRemoveTable(dbConn, "sqa_data")
#filterInput("sta_it_402/data/grades/1999_newhigher.csv", endYear = 1999, qualificationTypes = c("OrdinaryGrade", "NewHigher"))


#Map(filterInput, files_to_verify, startYear = 1989, endYear = 1999, qualificationTypes = "Higher")
#files_to_verify[sapply(files_to_verify, filterInput, startYear = 1989, endYear = 1999, qualificationTypes = "Higher")]


## (runPipeline("sta_it_402/data/grades/1989_Higher.csv", dbConn, TRUE))
## lapply(files_to_verify[sapply(files_to_verify, filterInput, startYear = 1989, qualificationTypes = "Higher")], 
##        runPipeline, dbConn) #, overwriteDataStore = TRUE)
## (removeDuplicatesFromDataStore(dbConn, "sqa_data"))


### Lag calculations - subject changes, change, growth rate in focus subject(s)

In [15]:
# note lag will take into account "duplicate" years such as 2015 with two strands of Higher as New Higher introduced
# to double-check this print out timeine and lag statement in loop

getSubjectChanges <-
    function(awardData, lag = 1, convertNullsToNa = FALSE) {
    
        subject_changes <- data.frame(qualificationPreviousYear = factor(), previousYear = factor(), 
                                      qualificationCurrentYear = factor(), currentYear = factor(), 
                                      countSubjectsPreviousYear = integer(), countSubjectsCurrentYear = integer(), 
                                      subjectsInCommon = character(), countSubjectsInCommon = integer(), 
                                      subjectsDropped = character(), subjectsAdded = character(), 
                                      countSubjectsDropped = integer(), countSubjectsAdded = integer(),
                                      stringsAsFactors = FALSE)
        subject_changes$subjectsInCommon <- list() # only way this works...
        subject_changes$subjectsDropped <-  list()
        subject_changes$subjectsAdded <- list()
        
        
        timeline <- awardData %>%
            arrange(year, qualification) %>%
            distinct(year, qualification)
         # print(str(timeline))
         # print(timeline)

        for (currentRow in 1:(nrow(timeline))) { 
            
            ## comment out the next three lines to check timeline being used to generate dataframe
            # previousYear <- as.factor(timeline$year[currentRow - lag])
            # currentYear <- as.factor(timeline$year[currentRow])
            # print(paste(lag, currentRow, previousYear, currentYear))        

            previousYear <- timeline$year[currentRow - lag]
            if ((currentRow - lag) > 0) {
                qualificationPreviousYear <- timeline$qualification[currentRow - lag]
                subjectsPreviousYear <- awardData %>%
                                          filter(year == previousYear) %>%
                                          select(Subject) %>%
                                          mutate_at(c("Subject"), as.character)
                countSubjectsPreviousYear <- nrow(subjectsPreviousYear)               
            } 
            
            currentYear <- timeline$year[currentRow]
            qualificationCurrentYear <- timeline$qualification[currentRow]
            subjectsCurrentYear <- awardData %>%
                                    filter(year == currentYear) %>%
                                    select(Subject) %>%
                                    mutate_at(c("Subject"), as.character)
            countSubjectsCurrentYear <- nrow(subjectsCurrentYear)


            if ((currentRow - lag) > 0) {
                subjectsInCommon <- intersect(subjectsPreviousYear, subjectsCurrentYear)
                countSubjectsInCommon <- lengths(subjectsInCommon) # sees length one if calculated when creating new row

                subjectsDropped <- setdiff(subjectsPreviousYear, subjectsCurrentYear)
                countSubjectsDropped <- lengths(subjectsDropped)

                subjectsAdded <- setdiff(subjectsCurrentYear, subjectsPreviousYear)
                countSubjectsAdded <- lengths(subjectsAdded)              
            } 


            
            if ((currentRow - lag) > 0) 
                subject_changes <- subject_changes %>%
                                    add_row(qualificationPreviousYear = qualificationPreviousYear, previousYear = previousYear, qualificationCurrentYear = qualificationCurrentYear, currentYear = currentYear,
                                            countSubjectsPreviousYear = countSubjectsPreviousYear, countSubjectsCurrentYear = countSubjectsCurrentYear, 
                                            subjectsInCommon = list(subjectsInCommon), countSubjectsInCommon = countSubjectsInCommon, 
                                            subjectsDropped =  list(subjectsDropped), subjectsAdded = list(subjectsAdded), 
                                            countSubjectsDropped = countSubjectsDropped, countSubjectsAdded = countSubjectsAdded)
            else
                subject_changes <- subject_changes %>%
                                    add_row(qualificationCurrentYear = qualificationCurrentYear, currentYear = currentYear,
                                            countSubjectsCurrentYear = countSubjectsCurrentYear)

        } # end for
        
        if (convertNullsToNa) 
            subject_changes <- subject_changes %>%
                                mutate(subjectsInCommon = na_if(subjectsInCommon, "NULL"), # does not recognise NULL without the quotes
                                       subjectsDropped = na_if(subjectsDropped, "NULL"),
                                       subjectsAdded = na_if(subjectsAdded, "NULL")
                                      )

        
        return(subject_changes)
    }

### Cleanup


In [16]:
# currently leaving DB connection detail in here but actually running where used

# dbDisconnect(dbConn)
# unlink(paste0(path_to_file_store, "/sqa-data-db.sqlite"))
